# This notebook enables the augmentation of lidar point clouds, adding fog

In [ ]:
import numpy as np
from tqdm import tqdm
from pathlib import Path
from helper.paths import paths
from classes.parameter_set import ParameterSet
from helper.fog_simulation import FogSimulation
from helper.pointcloud_io import load_pointcloud, write_pointcloud
from helper.file_lists import load_file_paths_from_dir


In [ ]:
CWD = Path.cwd()
DELFT_TRAINING = Path(paths["DELFT_TRAINING"])
ORIGINAL_PATH = CWD / "data" / "original"
FOGGY_PATH = CWD / "data" / "foggy"
FILE_LIST_KITTI = CWD / "file_lists" / "KITTI.txt"

color_feature = 2
color_dict = {
    0: "x",
    1: "y",
    2: "z",
    3: "intensity",
    4: "distance",
    5: "angle",
    6: "channel",
}
color_name = color_dict[color_feature]

# Default settings
noise = 10
noise_variant = "v4"
gain = True
p = ParameterSet(
    gamma=0.000001,
    gamma_min=0.0000001,
    gamma_max=0.00001,
    gamma_scale=10000000,
    
)
alpha = 0.006
p.set_params(alpha)

# Fog sim
INTEGRAL_PATH = CWD / "integral_lookup_tables" / "original"
AVAILABLE_TAU_Hs = [20]
RNG = np.random.default_rng(seed=42)
fog_sim = FogSimulation(INTEGRAL_PATH, AVAILABLE_TAU_Hs, RNG)


# KITTI settings
dataset = "KITTI"
min_value = -1
max_value = -1
num_features = 4
extension = "bin"
d_type = np.float32
# intensity_multiplier = 255
intensity_multiplier = 1 # For delft
color_dict[6] = "not available"


In [ ]:
file_list = load_file_paths_from_dir(data_path=DELFT_TRAINING/"velodyne")
for alpha in [0.06]: # Paper: 0.005, 0.01, 0.02, 0.03, 0.06
    print(f"Alpha: {alpha}")
    p.set_params(alpha)
    Path(DELFT_TRAINING/f"velodyne_fog_{alpha}").mkdir(exist_ok=True)
    for file_path in tqdm(file_list):
        file_name = file_path.split("/")[-1]
        current_pc = load_pointcloud(file_path, color_dict, color_feature, d_type, num_features, intensity_multiplier)
        pc, simulated_fog_pc, info_dict = fog_sim.simulate_fog(p, current_pc, noise, gain, noise_variant)
        write_pointcloud(pc, DELFT_TRAINING/f"velodyne_fog_{alpha}", file_name)

